# Example 1

This example aims to show the usage of the decisiorama tool for solving MCDA problems. This example is based on the paper of Lahtinen 2007[addlink] 



In [1]:
# import libraries 
import numpy as np
import decisiorama
from decisiorama.pda.preference import Objective, Evaluator
from decisiorama.pda import utility, aggregate, ranker
import itertools

## Definition of the solutions

Here we define the solutions. For each leaf objective, 

In [2]:
# Phosporous
p = np.array([[0.9, 1.1],
              [1.1, 1.3],
              [1.3, 1.7],
              [0.0, 0.0],
              [0.0, 0.0],
              [0.0, 0.0],
              [0.5, 0.6],
              [0.0, 0.0],
              [4.0, 4.8],])

# Nitrates
n2 = np.array([[0.09, 0.11],
               [0.09, 0.11],
               [0.14, 0.17],
               [0.00, 0.00],
               [0.00, 0.00],
               [0.00, 0.00],
               [0.00, 0.00],
               [0.00, 0.00],
               [0.40, 0.48],])

# Climate
climate = np.array([[0.0, 0.0],
                    [0.0, 0.0],
                    [0.5, 1.5],
                    [-1.5, -0.5],
                    [0.0, 0.0],
                    [0.5, 1.5],
                    [0.0, 0.0],
                    [0.0, 0.0],
                    [-2.5, -1.5],])

# Savings
savings = np.array([[1.8, 2.2],
                    [1.8, 2.2],
                    [1.8, 2.2],
                    [1.8, 2.2],
                    [0.9, 1.1],
                    [9.0, 11.0],
                    [32.0, 40.0],
                    [14.0, 18.0],
                    [3.5, 4.5],])

# Overall
overall = None

# Create of a dictionary with the solutions
sols = dict(p=p, n2=n2, climate=climate, savings=savings, overall=overall)

In [3]:
# Define the problems
costs = [1.0, 1.0, 2.0, 10.0, 8.0, 11.0, 43.0, 23.0, 20.0]
water = [0.03, 0.07, 0.04, 0.015, 0.10, 0.38, 0.15, 0.34, 0.46]
budget_limit = 45.0
n = 1000

In [4]:
p = Objective(
        name = 'p',
        w = 0.25,
        alternatives = sols['p'], 
        obj_min = 0.0, 
        obj_max = 4.8, 
        n = n, 
        utility_func = utility.exponential, 
        utility_pars = [0.0, ], 
        aggregation_func = aggregate.additive, 
        aggregation_pars = None,
        maximise = False)

n2 = Objective(
        name = 'n2',
        w = 0.25,
        alternatives = sols['n2'], 
        obj_min = 0.0, 
        obj_max = 0.48, 
        n = n, 
        utility_func = utility.exponential, 
        utility_pars = [0.0, ], 
        aggregation_func = aggregate.additive, 
        aggregation_pars = None,
        maximise = False)

climate = Objective(
        name = 'climate',
        w = 0.25,
        alternatives = sols['climate'], 
        obj_min = -2.5, 
        obj_max = 1.5, 
        n = n, 
        utility_func = utility.exponential, 
        utility_pars = [0.0, ], 
        aggregation_func = aggregate.additive, 
        aggregation_pars = None,
        maximise = False)

savings = Objective(
        name = 'savings',
        w = 0.25,
        alternatives = sols['savings'], 
        obj_min = 0.9, 
        obj_max = 40.0, 
        n = n, 
        utility_func = utility.exponential, 
        utility_pars = [0.0, ], 
        aggregation_func = aggregate.additive, 
        aggregation_pars = None,
        maximise = False)

overall = Objective(
        name = 'overall',
        w = 0.25,
        alternatives = sols['climate'], 
        obj_min = 0.9, 
        obj_max = 40.0, 
        n = n, 
        utility_func = utility.exponential, 
        utility_pars = 0.0, 
        aggregation_func = aggregate.additive, 
        aggregation_pars = None,
        maximise = False)



In [5]:
overall.add_children(p)
overall.add_children(n2)
overall.add_children(climate) 
overall.add_children(savings)




In [6]:
prob = dict(p=p, n2=n2, climate=climate, savings=savings, 
            overall=overall)

x = [1,1,1,1,1,1,1,1,1]
overall.get_value(x)


array([6.62595908, 6.10187553])

In [7]:
def filter_inps(inps):
    out = []
    def follow_up(pred, post):
        if post:
            if not pred:
                return False
        return True
        
    def mutual_exclusive(a, b):
        if a and b:
            return False
        return True

    for x in inps:        
    
        # follow up action
        if not follow_up(x[3], x[4]):
            continue
        
        # Mutually exclusive actions
        if not mutual_exclusive(x[3], x[5]):
            continue
        
        if not mutual_exclusive(x[6], x[7]):
            continue
        
        if not mutual_exclusive(x[6], x[8]):
            continue
     
        # Budget and water constraints
        budget = np.sum([a for i, a in enumerate(costs) if x[i]]) 
        if budget > budget_limit:
            continue
    
        water_target = 1.0 - np.prod([(1.0 - a) for i, a in enumerate(water) if x[i]])
        if water_target < 0.5:
            continue
        
        out.append(x)
    return out
# get all the results
inp_comb = list(itertools.product([0, 1], repeat=len(x)))

inps = np.array(filter_inps(inp_comb))
res = np.array(list(map(overall.get_value, inps)))



In [8]:
ee = Evaluator(inps, res)
ee.add_function(ranker.mean, minimize=False)
ee.add_function(ranker.iqr, minimize=True)
# ee.get_pareto_solutions()

In [10]:
ee.get_pareto_solutions()

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36], dtype=int64)